
# Pretrained Model
## Imports

In [1]:
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
import pandas as pd
import numpy

In [2]:
# The directory where we will store our smaller dataset/
base_dir = "data/lab_01_split"

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')


Homer = 'homer_simpson'
Bart = 'bart_simpson'
Burns = 'charles_montgomery_burns'
Krusty = 'krusty_the_clown'
Lisa = 'lisa_simpson'
Milhouse = 'milhouse_van_houten'
Marge = 'marge_simpson'
Moe = 'moe_szyslak'
Ned = 'ned_flanders'
Principal = 'principal_skinner'


# Directory with our training images
train_homer = os.path.join(train_dir, Homer)
train_bart = os.path.join(train_dir, Bart)
train_burns = os.path.join(train_dir, Burns)
train_krusty = os.path.join(train_dir, Krusty)
train_lisa = os.path.join(train_dir, Lisa)
train_milhouse = os.path.join(train_dir, Milhouse)
train_marge = os.path.join(train_dir, Marge)
train_more = os.path.join(train_dir, Moe)
train_ned = os.path.join(train_dir, Ned)
train_principal = os.path.join(train_dir, Principal)

# Directory with our valiation images
val_homer = os.path.join(validation_dir, Homer)
val_bart = os.path.join(validation_dir, Bart)
val_burns = os.path.join(validation_dir, Burns)
val_krusty = os.path.join(validation_dir, Krusty)
val_lisa = os.path.join(validation_dir, Lisa)
val_milhouse = os.path.join(validation_dir, Milhouse)
val_marge = os.path.join(validation_dir, Marge)
val_more = os.path.join(validation_dir, Moe)
val_ned = os.path.join(validation_dir, Ned)
val_principal = os.path.join(validation_dir, Principal)

# Directory with our test images
test_homer = os.path.join(test_dir, Homer)
test_bart = os.path.join(test_dir, Bart)
test_burns = os.path.join(test_dir, Burns)
test_krusty = os.path.join(test_dir, Krusty)
test_lisa = os.path.join(test_dir, Lisa)
test_milhouse = os.path.join(test_dir, Milhouse)
test_marge = os.path.join(test_dir, Marge)
test_more = os.path.join(test_dir, Moe)
test_ned = os.path.join(test_dir, Ned)
test_principal = os.path.join(test_dir, Principal)


In [4]:
print('total training Homer images:', len(os.listdir(train_homer)))
print('total validation Homer images:', len(os.listdir(val_homer)))
print('total test Homer images:', len(os.listdir(test_homer)))

total training Homer images: 1572
total validation Homer images: 337
total test Homer images: 337


# 2. Preprocessing


In [33]:
train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

In [34]:
train_generator = train_datagen.flow_from_directory(
        train_dir, # target directory
        class_mode='categorical')

Found 9663 images belonging to 10 classes.


In [35]:
validation_generator = validation_datagen.flow_from_directory(
        validation_dir, # validation directory
        class_mode='categorical')

Found 2072 images belonging to 10 classes.


In [36]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (32, 256, 256, 3)
labels batch shape: (32, 10)


# 3. Building the Model



In [37]:
base_resnet50 = keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape = (256, 256, 3))


In [38]:
# freeze the conv-net structure
base_resnet50.trainable = False



In [39]:


custom_model = keras.Sequential(
    [
        keras.Input(shape=(256, 256, 3)),
        base_resnet50,
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation='softmax'),
    ], name='global_model'
)


custom_model.summary()

Model: "global_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten_4 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1310730   
Total params: 24,898,442
Trainable params: 1,310,730
Non-trainable params: 23,587,712
_________________________________________________________________


# 5. Training the model

In [40]:
earlystop = EarlyStopping(
    monitor='val_categorical_accuracy',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto')

csv_logger = CSVLogger('training.log',
                       separator=',',
                       append=False)

In [41]:
custom_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.8),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

Fitting the model

In [43]:
history = custom_model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[earlystop, csv_logger]
)



Epoch 1/5
100/100 [==============================] - 368s 4s/step - loss: 5959.0430 - categorical_accuracy: 0.6628 - val_loss: 5834.0879 - val_categorical_accuracy: 0.6706